# Multi-Cluster Visibility

## Prerequisites

- Cluster Controller installed to K8s clusters.
- The `cnatix` CLI is installed.
- The user who runs the test plan is an admin of the root org.
- `cnatix login` has already run.

To verify that a user is an admin of the root org, run:

In [ ]:
# Verify that the role binding is set to the <root-org>:admin.
cnatix iam users self

## Test case 1: Cost, efficiency, waste visibility for K8s environments by clusters and mapping to orgs

### Expected Results

- A user is able to create orgs.
- A user is able to map the orgs to K8s clusters.
- A user is able to view cost, efficiency, and waste per K8s cluster/org.

### Test Execution Steps

#### *Step 1*: Create orgs

Create new orgs whose parent org is the root org.

To create a new org, run:

In [ ]:
# Verify the current list of orgs.
cnatix iam orgs list

In [ ]:
ROOT_ORG_UUID=<UUID of root org>
ORG_NAME=<name of the new org>
ORG_DESCRIPTION=<description of the org>

cnatix iam orgs create \
  --name "${ORG_NAME}" \
  --parent-org-uuid ${ROOT_ORG_UUID} \
  --description "${ORG_DESCRIPTION}"

# Verify that the new org is created.
cnatix iam orgs list

#### *Step 2*: Map a K8s cluster to an org

First, get a cluster UUID of a target K8s cluster. Run:

In [ ]:
cnatix clusters list

Then create a conversion rule to map a K8s cluster to an org created in the previous step.

In [ ]:
ORG_UUID=<UUID of org where a K8s cluster is assigned>
CLUSTER_UUID=<Cluster UUID>

cnatix infrastructure compute instances convert create \
  --kind org-allocation \
  --target "${ORG_UUID}" \
  --label-filters=cluster-uuid="${CLUSTER_UUID}"

# Verify the org allocation
cnatix infrastructure compute instances list

The following command is a shortcut version:

In [ ]:
cnatix clusters update --name "${CLUSTER_NAME}" --owner-org "${ORG_NAME}"

# Verify the org allocation
cnatix infrastructure compute instances list

#### *Step 3*: View Cost, efficiency, waste visibility

Go to https://app.cloudnatix.com. Use the org selector to view cost, efficiency, and waste per K8s cluster/org.

## Test case 2: Cost, efficiency, waste visibility for specific categories (e.g., AWS accounts, AWS VPC, Azure subscriptions, resource groups)

### Expected Results

- A user is able to view cost, efficiency, and waste per certain category (e.g., Azure subscriptions, AWS accounts)

### Test Execution Steps

Go to https://app.cloudnatix.com. You can select a grouping key from the "Group by" box and
view the breakdown of cost, efficiency, and waste.

For Azure:

- Select `azure/subscription-id` to group resources by Azure subscriptions.
- Select `azure/resource-group` to group resources by Azure resource groups.

For AWS:

- Select `account-id` to group resources by AWS account IDs.

## Test case 3: K8s workload-based cost allocation/visibility

### Expected Results

- A user is able to see the cost associated with workloads when workloads have resource requests/limits.

### Test Execution Steps

Visit https://app.cloudnatix.com/app/workloads and check the "Cost" column in the workload table.

## Test case 4: K8s namespace based cost allocation/visibility

### Expected Results

- A user is able to see the cost associated with each namespace.

### Test Execution Steps

Run:

In [ ]:
cnatix k8s cost

The output shows the following information per cluster/namespace.

- Cost per hour for workloads that have resource requests/limits
- Number of workloads that have resource requests/limits
- Number of workloads that don't have resource requests/limits

You can pass `--cluster-name <cluster-name>` `--namespace <namespace-regexp>` or `--labels <key1=value1,key2=value2,...>` to the command to
filter the output. Here is an example that shows the cost of `kube-.*` namespaces across multiple clusters. This should return all resources matching
the namespace starting with `kube-` (e.g. `kube-system`).

> Both --cluster-name and --namespace parse the input as a regex.

In [ ]:
cnatix k8s cost --namespace "kube-.*"

## (Baremetal) Test case 5: Setting Machine Default Pricing

For baremetal environments, users can set up an environment that reflects the cost of running their instances.

### Expected Results

-  A user is able to change the pricing model for their baremetal instances.

### Test Execution Steps
Users can use the CLI command `infrastructure compute instances default-pricing create` with the flags `--cpu-core-cost`, `--mem-gb-cost`, and `--misc-cost`
to provide a default calculation for running their compute instances.

> The costs are in dollars per hour.

In [ ]:
cnatix infrastructure compute instances default-pricing create \
  --cpu-core-cost 0.05 \
  --mem-gb-cost 0.04 \
  --misc-cost 0.002

For example, for an instance with `4` CPU cores, `8` Gb memory, the cost `($/hr)` will be calculated
as `4 CPUCore * CPUCoreCost + 8 MemGb * MemGbCost + MiscCost`. The updates will be reflected on the dashboard after some time.

> By default, we create three machine families (instances) for baremetal environments. Users can also add their own machine families with
> `cnatix infrastructure compute instances machine-family create`